# Part 3 Preprocessing

Import Module

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

/Users/tengfone/.pyenv/versions/3.8.5/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Pathings

In [51]:
TF_train_pathing = "/Users/tengfone/Desktop/ML_Project/data/CN/train"
TF_test_pathing = "/Users/tengfone/Desktop/ML_Project/data/CN/dev.in"

Loading Files

In [52]:
def open_train_file(file_path):
    with open(file_path) as f_lines:
        all_datas = f_lines.read().splitlines()
        all_datas[:] = [x for x in all_datas if x]

    word_tags = list()
    output_word_tags = list()
    for i in all_datas:
        i = i.split(' ')
        word_tags.append(i)
        
    for i in word_tags:
        if(len(i) == 3):
            i = [i[0]+i[1],i[2]]
            output_word_tags.append(i)
        else:
            output_word_tags.append(i)
    return output_word_tags

In [53]:
def open_test_file(file_path):
    with open(file_path) as f_lines:
        all_datas = f_lines.read().splitlines()

    output = list()
    para = 0

    for i in all_datas:
        if i != "":
            output.append([i, para])
        else:
            para += 1

    return output

Convert to DF

In [54]:
def to_train_df(word_tags):
    df = pd.DataFrame(word_tags, columns=['Text', 'Labels'])
    return df

In [55]:
def to_test_df(list_words):
    df = pd.DataFrame(list_words, columns=['Text', 'Paragraph'])
    return df

Emission Part a

In [56]:
def emission(train_data):
    # Total Number of Labels (DF)
    df_num_labels = train_data['Labels'].value_counts().rename_axis('Labels').reset_index(name='CountY')

    # Total Number of y -> x
    df_emission = train_data.groupby(["Text","Labels"]).size().reset_index()
    df_emission.columns = ["Text", "Labels", "CountY->X"]

    # e(x|y)
    emission_output = pd.merge(df_emission,df_num_labels, on = "Labels")
    emission_output["Emission"] = emission_output["CountY->X"]/emission_output["CountY"]

    return emission_output

Emission Part b (Fix with UNK)

In [57]:
def replace_unk(word,list_of_words):
    if(word not in list_of_words):
        return "#UNK#"
    else:
        return word

In [58]:
def emission_fix(train_data,test_data,k):
    list_of_words = train_data['Text'].to_list()
    test_data['Text'] = test_data['Text'].apply(lambda x: replace_unk(x,list_of_words))
  
  # Total Number of Labels (DF)
    df_num_labels = train_data['Labels'].value_counts().rename_axis('Labels').reset_index(name='CountY')

  # Total Number of y -> x
    df_emission = train_data.groupby(["Text","Labels"]).size().reset_index()
    df_emission.columns = ["Text", "Labels", "CountY->X"]

  # e(x|y)
    emission_output = pd.merge(df_emission,df_num_labels, on = "Labels")

  # Unique Labelsoutlist.append(output)
    unique_labels = emission_output.Labels.unique().tolist()
    print(emission_output)

    for i in range(len(unique_labels)):
        each_label = unique_labels[i]
        print(f"Running at {each_label}")
        count_y = emission_output[emission_output['Labels']==each_label]['CountY'].values[0]
        temp_list = list()
        for index, row in test_data.iterrows():
            word = row['Text']
            if (word == "#UNK#"):
                output = k/ (count_y + k)
                temp_list.append(output)
            else:
                count_yx = 0
                a = emission_output[(emission_output['Text'] == word)]
                a2 = a[(a['Labels'] == each_label)]
                # a = df.query('Text==@word and Labels==@label')
                if (a2.size==0):
                    output = 0
                    temp_list.append(output)
                else:
                    a2 = a2['CountY->X'].values[0]
                    count_yx = a2
                    output = count_yx/(count_y + k)
                    temp_list.append(output)
        test_data[each_label] = temp_list
                    
    return test_data

Get new Fixed Emission

In [59]:
k = 0.5
# Training
train_data = open_train_file(TF_train_pathing)
train_data_df = to_train_df(train_data)
emitted = emission(train_data_df)

# Testing
test_data = open_test_file(TF_test_pathing)

test_data_df = to_test_df(test_data)
emitted_fix = emission_fix(train_data_df,test_data_df,k)

      Text      Labels  CountY->X  CountY
0        !   I-neutral          1    3507
1        #   I-neutral         19    3507
2        %   I-neutral          3    3507
3        &   I-neutral          2    3507
4        '   I-neutral          3    3507
...    ...         ...        ...     ...
19862   闺蜜  I-negative          1     135
19863   青山  I-negative          1     135
19864   高危  I-negative          1     135
19865   高手  I-negative          1     135
19866    鸡  I-negative          2     135

[19867 rows x 4 columns]
Running at I-neutral
Running at O
Running at B-negative
Running at B-neutral
Running at B-positive
Running at I-positive
Running at I-negative


Formatting Dataframe to Text/max_label/Paragraph

In [60]:
emitted_fix

,Text,Paragraph,I-neutral,O,B-negative,B-neutral,B-positive,I-positive,I-negative
0,十年,0,0.000000,0.000114,0.000000,0.000000,0.000000,0.000000,0.00000
1,前,0,0.000285,0.000318,0.000000,0.000000,0.000000,0.000723,0.00000
2,马云,0,0.000000,0.000000,0.000000,0.000618,0.001624,0.000000,0.00000
3,告诉,0,0.000000,0.000216,0.000000,0.000000,0.000000,0.000000,0.00000
4,你,0,0.002851,0.009045,0.018083,0.006484,0.004060,0.001447,0.00738
...,...,...,...,...,...,...,...,...,...
13065,、,343,0.001996,0.007267,0.000000,0.000000,0.000000,0.000000,0.00000
13066,#UNK#,343,0.000143,0.000006,0.001808,0.000154,0.000406,0.000362,0.00369
13067,180,343,0.000000,0.000038,0.000000,0.000000,0.000000,0.000000,0.00000
13068,公里,343,0.000285,0.000267,0.000000,0.000000,0.000000,0.000000,0.00000


# SG_Dataset

In [50]:
export_df = emitted_fix.drop(['Text', 'Paragraph'], axis=1)
export_df = export_df[['O','B-neutral','I-neutral','B-positive','I-positive','B-negative','I-negative']]
export_df = export_df.T
print(export_df)
out_np = export_df.to_numpy()
print(out_np)
np.savetxt("SG_np.csv", out_np, delimiter=",")

               0         1         2         3         4         5      \
O           0.000006  0.000045  0.000376  0.006803  0.014166  0.000003   
B-neutral   0.000000  0.000000  0.000000  0.000000  0.000603  0.000020   
I-neutral   0.000000  0.000000  0.000000  0.000104  0.003442  0.000017   
B-positive  0.000000  0.000000  0.000000  0.000000  0.001089  0.000136   
I-positive  0.000000  0.000000  0.000000  0.003909  0.006645  0.000195   
B-negative  0.000000  0.000000  0.000000  0.000000  0.000878  0.000439   
I-negative  0.000000  0.000000  0.000000  0.000000  0.003988  0.000997   

               6         7         8         9      ...     34181     34182  \
O           0.000174  0.000028  0.023826  0.000118  ...  0.000003  0.000006   
B-neutral   0.000040  0.000000  0.000000  0.000000  ...  0.000020  0.000000   
I-neutral   0.000070  0.000000  0.005528  0.000000  ...  0.000017  0.000000   
B-positive  0.000817  0.000000  0.000000  0.000000  ...  0.000136  0.000000   
I-positive  

# EN_Dataset

In [39]:
export_df = emitted_fix.drop(['Text', 'Paragraph'], axis=1)
export_df = export_df[['B-NP','I-NP','B-VP','B-ADVP','B-ADJP','I-ADJP','B-PP','O','B-SBAR','I-VP','I-ADVP','B-PRT','I-PP','B-CONJP','I-CONJP','B-INTJ','I-INTJ','I-SBAR','B-UCP','I-UCP','B-LST']]
export_df = export_df.T
print(export_df)
out_np = export_df.to_numpy()
print(out_np)
np.savetxt("EN_np.csv", out_np, delimiter=",")

            0         1         2         3         4         5         6      \
B-NP     0.000085  0.000000  0.000085  0.000000  0.000233  0.000000  0.000042   
I-NP     0.000018  0.000000  0.000311  0.001575  0.000275  0.016376  0.000073   
B-VP     0.000000  0.032144  0.000000  0.083400  0.000000  0.000000  0.000000   
B-ADVP   0.000000  0.000000  0.000561  0.000841  0.000000  0.000000  0.000000   
B-ADJP   0.000000  0.000000  0.003426  0.000000  0.000000  0.000000  0.000000   
I-ADJP   0.000000  0.000000  0.001741  0.001741  0.000000  0.000000  0.000000   
B-PP     0.000000  0.000000  0.000000  0.100394  0.000000  0.000000  0.000000   
O        0.000000  0.000000  0.000042  0.000796  0.000000  0.000209  0.000000   
B-SBAR   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
I-VP     0.000000  0.000394  0.001870  0.092327  0.000000  0.000000  0.000000   
I-ADVP   0.000000  0.000000  0.002751  0.013755  0.000000  0.000000  0.000000   
B-PRT    0.000000  0.000000 

# CN_Dataset

In [61]:
export_df = emitted_fix.drop(['Text', 'Paragraph'], axis=1)
export_df = export_df[['O','B-neutral','I-neutral','B-positive','I-positive','B-negative','I-negative']]
export_df = export_df.T
print(export_df)
out_np = export_df.to_numpy()
print(out_np)
np.savetxt("CN_np.csv", out_np, delimiter=",")

               0         1         2         3         4         5      \
O           0.000114  0.000318  0.000000  0.000216  0.009045  0.061742   
B-neutral   0.000000  0.000000  0.000618  0.000000  0.006484  0.000000   
I-neutral   0.000000  0.000285  0.000000  0.000000  0.002851  0.004277   
B-positive  0.000000  0.000000  0.001624  0.000000  0.004060  0.000000   
I-positive  0.000000  0.000723  0.000000  0.000000  0.001447  0.000000   
B-negative  0.000000  0.000000  0.000000  0.000000  0.018083  0.000000   
I-negative  0.000000  0.000000  0.000000  0.000000  0.007380  0.000000   

               6         7         8         9      ...     13060     13061  \
O           0.000051  0.000006  0.000800  0.000203  ...  0.000267  0.007267   
B-neutral   0.000000  0.000154  0.000000  0.000000  ...  0.000000  0.000000   
I-neutral   0.000000  0.000143  0.000285  0.000000  ...  0.000285  0.001996   
B-positive  0.000000  0.000406  0.000000  0.000000  ...  0.000000  0.000000   
I-positive  